# 01 - Single Inheritance

Python supports multiple inheritance, unlike languages like Java which only support single inheritance (but allows multiple interfaces), but we'll look at that later.

#### Some basics

If our classes tend to form a natural hierarchy, then creating an inheritance tree may be useful.

Classes lower down in the hierarchy will **inherit** characteristics (state and behaviour i.e properties and methods) from those higher up. 

But they can also **extend** to have characteristics that those higher up didn't have.

They can also **override** characteristics in those higher up.

**Inherits from / subclasses / is a child of / derives from** are all synonymous terms. Note that the terms **child** and **parent** refer to direct relationships while **ancestor** refers to indirect relationships. So if **A** inherits from **B** which inherits from **C**, then **C** is *not* a parent of **A** but it *is* an ancestor.

#### `isinstance` vs `type` vs `issubclass`

If `Student` inherits from `Person`, then any instances of `Student` are automatically instances of `Person`. But, `Person` instances are of course not `Student` instances.

`isinstance` does not look at direct relationships; since all objects inherit from `object`, `isinstance(<anything>, object)` will always be `True`.

The same applies for `issubclass` but **issubclass** can only be used to inspect inheritance relationships between **classes** not instances.

`type` returns the class that was used to create the instance - it does not look at inheritance.

In [2]:
class Person:
    pass

class Student(Person):
    pass

s1 = Student()

isinstance(s1, Person)

True

In [3]:
class CollegeStudent(Student):
    pass

issubclass(CollegeStudent, Person)

True

One useful thing to remember is that the types printed out using `type` aren't necessarily the builtin objects themselves but rather a string representation. But the actual types can be found in other modules. Here are two examples below:

In [16]:
import types

def my_func():
    pass

types.FunctionType is type(my_func)

True

In [17]:
import math

types.ModuleType is type(math)

True

# 02 - The object Class

Despite being lowercase, `object` is a class, not an instance of some other class.

If one of our classes does not override a characteristic from `object` when it inherited it, then that characteristic will be **identical** to the one found in our class. For example, if we do not implement `__init__` method, then the `__init__` method of `object` will be called.

In [18]:
class Person:
    pass

Person.__init__ is object.__init__

True

But instances will technically have a different `__init__`.

In [26]:
p1 = Person()

p1.__init__, Person.__init__, object.__init__

(<method-wrapper '__init__' of Person object at 0x0000019FDF8A12A0>,
 <slot wrapper '__init__' of 'object' objects>,
 <slot wrapper '__init__' of 'object' objects>)

# 03 - Overriding

When we inherit from another class, we inherit its attributes, including all callables. We can choose to redefine an existing callable in the subclass - this is known as **overriding**.

When it comes to calling a method of our class, we first look to see we've overridden it within the class. If not, we go up the inheritance tree to see if it can be found there, eventually ending up at the `object` level. If it's not found there, we'll get an error.

**Tip**

- Objects have a property: `__class__` -> returns the **class** the object was created from (but we should use `type()` instead; see later).
- Classes have a propery: `__name__` -> returns a **string** containing the name of the class.

If we want to get the name of the class used to create the object -> `object.__class__.__name__`

Here's an example to demonstrate an important point:

In [1]:
class Person:
    def eat(self):
        print('Person eats')

    def work(self):
        print('Person works')

    def sleep(self):
        print('Person sleeps')

    def routine(self):
        self.eat()
        self.work()
        self.sleep()

class Student(Person):
    def work(self):
        print('Student studies')

In [2]:
s = Student()
s.routine()

Person eats
Student studies
Person sleeps


**Methods called from an instance are always bound to that instance**

- Above we see `routine` being called first - this is **bound** to the `Student` instance (so `self` is a `Student` object).
- Since `routine` doesn't exist in `Student`, we look up the inheritance tree, find it in `Person` and call it.
- Again, since `routine` is called and therefore bound to `s`, the `self` in `routine(self)` method refers to the instance of `Student`.
- So `self.work()` will look for `work` in `s` first which it finds..

# 04 - Extending

Extending is used for creating a more specialised subclass. 

Here's a similar example to the above where the generic `Person` class has some defined functionality, but the `Student` class has extended on top of it:

In [3]:
class Person:
    def eat(self):
        print('Person eats')

    def sleep(self):
        print('Person sleeps')

    def routine(self):
        self.eat()
        self.study()  # not defined in this class
        self.sleep()

class Student(Person):
    def study(self):
        print('Student studies')

In [4]:
s = Student()
s.routine()

Person eats
Student studies
Person sleeps


Of course, we'll have a problem if we call `p.routine()`, but sometimes it may be that the `Person` class is intended to be interacted with via inheritance **only**. 

These types of classes are known as **abstract base classes (abc)**.

In these classes, it will generally provide some generic behaviour (`eat` and `sleep`) but will expect all inheritors to implement some specific behaviour (`study`). 

#### Example

Here is an important example to highlight the differences in behaviour with class attributes vs instance attributes:

In [13]:
class Account:
    apr = 3.0
    def __init__(self):
        self.account_type = 'Generic Account'
        
    def calc_interest(self):
        return f'Calc interest on {self.account_type} with APR = {self.apr}'

In [14]:
class Savings(Account):
    apr = 5.0
    
    def __init__(self):
        self.account_type = 'Savings Account'

In [17]:
a = Account()
a.calc_interest()

'Calc interest on Generic Account with APR = 3.0'

In [18]:
s = Savings()
s.calc_interest()

'Calc interest on Savings Account with APR = 5.0'

**Both** of these worked because `self.apr` did not exist as an instance attribute so we looked at the bound instance's class attribute.

Had we used `Account.apr` or `Savings.apr` instead of `self.apr`, one of the two would give us the wrong number.

The issue with this approach is someone could define `self.apr = 500` and this would take precedence in `calc_interest()`:

In [19]:
s = Savings()
s.apr = 500
s.calc_interest()

'Calc interest on Savings Account with APR = 500'

What we need instead of either of these is to find the `apr` attribute of the class that this instance was created from. We can use `type(self)` (or slightly worse `self.__class__`) for this very purpose:

In [20]:
class Account:
    apr = 3.0
    def __init__(self):
        self.account_type = 'Generic Account'
        
    def calc_interest(self):
        return f'Calc interest on {self.account_type} with APR = {type(self).apr}'

In [21]:
class Savings(Account):
    apr = 5.0
    
    def __init__(self):
        self.account_type = 'Savings Account'

In [22]:
a = Account()
a.calc_interest()

'Calc interest on Generic Account with APR = 3.0'

In [23]:
s = Savings()
s.calc_interest()

'Calc interest on Savings Account with APR = 5.0'

# 05 - Delegating to Parent

This is an important subsection. Consider the following code:

In [24]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Student(Person):
    def __init__(self, name, age, major):
        self.name = name
        self.age = age
        self.major = major

Here we've overridden the `__init__` method, but we've had to copy a large chunk of the `__init__` code into our subclass. This goes against OOP where we try to use OOP principles to reduce repeated code. 

Instead, we should **delegate** back to the parent class for all the generics -> `super()` returns a **proxy object** - "an object that delegates calls to the correct class methods without making an additional object in order to do so". 

**But note:** the `self` argument of the parent method is *still* the instance that the method was **bound** to.

This allows us to run the method in the parent class *as* it's defined in the parent class, even if we have an override in place.

In [45]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print(f"self is an instance of {type(self).__name__} class")

class Student(Person):
    def __init__(self, name, age, major):
        super().__init__(name, age)
        self.major = major

In [46]:
s = Student('John', 20, 'math')
s.name, s.age, s.major

self is an instance of Student class


('John', 20, 'math')

In general, you should always **delegate first** (call `super()` before doing anything). 

This is because calling `super()` later might overwrite stuff you did just before.

# 06 - Slots

# 07 - Slots and Single Inheritance